#### Comparison of performance of Bayesian Neural Network using Varitional Inference and HMC 

### Import Libraries

In [1]:
using CSV
using DataFrames
using StatsBase
using Flux
using Distributions, Random
using Turing
using Metrics
using Dates

### Data

In [2]:
file_path = raw"C:\Users\khaitan\Downloads\iris_data\iris.csv"

"C:\\Users\\khaitan\\Downloads\\iris_data\\iris.csv"

In [3]:
df = CSV.File(file_path, header=["f1","f2","f3","f4", "class"]) |> DataFrame
first(df,5)

,f1,f2,f3,f4,class
,Float64,Float64,Float64,Float64,String15
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa


### Shuffling the data

In [4]:
df = df[shuffle(1:end), :]
first(df,5)

,f1,f2,f3,f4,class
,Float64,Float64,Float64,Float64,String15
1,5.1,3.3,1.7,0.5,Iris-setosa
2,4.7,3.2,1.6,0.2,Iris-setosa
3,6.9,3.1,5.4,2.1,Iris-virginica
4,6.3,3.4,5.6,2.4,Iris-virginica
5,7.7,3.0,6.1,2.3,Iris-virginica


### Label Encoding

In [5]:
ux = unique(df.class)
D = Dict("Iris-setosa" => 0, "Iris-versicolor" => 1, "Iris-virginica" => 2)

Dict{String, Int64} with 3 entries:
  "Iris-virginica"  => 2
  "Iris-setosa"     => 0
  "Iris-versicolor" => 1

In [6]:
int(x) = floor(Int, x)
df.label = [0 for i in 1:size(df, 1)]
for i in 1:length(df.class)
    df.label[i] = D[df.class[i]]
end;

### One-Hot Encoding

In [7]:
df.encoded_label = [[0, 0, 0] for i in 1:size(df, 1)]
for i in 1:length(df.label)
    df.encoded_label[i][int(df.label[i]) + 1] = 1
end;

In [8]:
first(df, 5)

,f1,f2,f3,f4,class,label,encoded_label
,Float64,Float64,Float64,Float64,String15,Int64,Array…
1,5.1,3.3,1.7,0.5,Iris-setosa,0,"[1, 0, 0]"
2,4.7,3.2,1.6,0.2,Iris-setosa,0,"[1, 0, 0]"
3,6.9,3.1,5.4,2.1,Iris-virginica,2,"[0, 0, 1]"
4,6.3,3.4,5.6,2.4,Iris-virginica,2,"[0, 0, 1]"
5,7.7,3.0,6.1,2.3,Iris-virginica,2,"[0, 0, 1]"


### Train-test split

In [13]:
X = reduce(vcat,transpose.([df.f1, df.f2, df.f3, df.f4]))
y = df.encoded_label

split_fraction = 0.75 
index = int(length(y)*split_fraction)

X_train, X_test = X[:, 1:index], X[:, index + 1:size(df, 1)];
y_train, y_test = y[1:index], y[index + 1:size(df, 1)];

In [10]:
println("Size of train set: ", size(X_train))
println("Size of train labels: ", size(y_train))
println("Size of test set: ", size(X_test))
println("Size of test labels: ", size(y_test))

Size of train set: (4, 112)
Size of train labels: (112,)
Size of test set: (4, 38)
Size of test labels: (38,)


### Define Network & Likelihood

In [11]:
neural_network = Chain(Dense(4,10,σ), 
                        #Dense(5,5,relu), 
                        Dense(10,3), 
                        softmax)

Chain(
  Dense(4 => 10, σ),                    # 50 parameters
  Dense(10 => 3),                       # 33 parameters
  NNlib.softmax,
)                   # Total: 4 arrays, 83 parameters, 588 bytes.

In [12]:
struct Likelihood
    network
end

Flux.@functor Likelihood #tell Flux to look for trainable parameters in Likelihood
(p::Likelihood)(x) = Multinomial.(1, [p.network(x)[:,i] for i in 1:size(x, 2)])

In [13]:
likelihood = Likelihood(neural_network)

params, likelihood_reconstructor = Flux.destructure(likelihood)
n_weights = length(params)
println("No of weight parameters: ", n_weights)

No of weight parameters: 83


### Prior Distribution of Weights

In [14]:
weight_prior = MvNormal(zeros(n_weights), ones(n_weights))

DiagNormal(
dim: 83
μ: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)


In [15]:
### Verfiy the network output
neural_network(X)

3×150 Matrix{Float64}:
 0.106954  0.104326  0.105824  0.0986519  …  0.0980669  0.10163   0.102563
 0.375457  0.480985  0.373805  0.489617      0.526568   0.474298  0.472911
 0.517589  0.414689  0.520371  0.411731      0.375365   0.424073  0.424526

In [16]:
likelihood_reconstructor(rand(weight_prior))(X)[1]

Multinomial{Float64, Vector{Float64}}(n=1, p=[0.007249307221910489, 0.02229786632171394, 0.9704528264563755])

### Define the model for inference

In [18]:
@model function TuringModel(likelihood_conditional, weight_prior, X, y)
    
    weights ~ weight_prior

    predictions = likelihood_reconstructor(weights)(X)
    
    for i in 1:length(y)
        y[i] ~ predictions[i]
    end
    
end

TuringModel (generic function with 2 methods)

### Posterior Samples using HMC

In [24]:
Random.seed!(123)
N = 500

start_t = now()

## We need to provide leapfrog step size, leapfrog step numbers 
ch = sample(TuringModel(likelihood_reconstructor, weight_prior, X_train , y_train), HMC(0.025, 10), N);
weights = MCMCChains.group(ch, :weights).value #get posterior MCMC samples for network weights

end_t = now()
time_taken = (end_t - start_t) 
println("Total  Time taken: ", Dates.value(time_taken)/1000, " seconds")

Sampling: 100%|█████████████████████████████████████████| Time: 0:31:39


Total  Time taken: 1907.06 seconds


In [25]:
final_weights = dropdims(mean(weights, dims = 1), dims=1);

### Accuracy

In [26]:
### Function to compute accuracy and get the wrong predictions

function accuracy(labels, predictions)
    
    acc = 0
    wrong_predictions = []
    for i in 1:length(labels)

        if labels[i] == argmax(predictions[i].p) - 1
            acc += 1
        else
            append!(wrong_predictions, [[int(labels[i]), argmax(predictions[i].p) - 1, predictions[i].p]])
        end
    end
    
    acc = acc / length(labels)
    
    return acc, wrong_predictions
end

accuracy (generic function with 1 method)

### Test Accuracy & Wrong Predictions

In [27]:
test_labels = df.label[index + 1:size(df, 1)];
test_preds = likelihood_reconstructor(vec(final_weights))(X_test)
test_acc, wrong_preds = accuracy(test_labels, test_preds)
println("Accuracy on Test Data: ", test_acc)

println("\nWrong Predictions :")
for i in 1:length(wrong_preds)
    
    println("Actual : ", wrong_preds[i][1], "  Prediction : ", wrong_preds[i][2], "  Probability Scores :", wrong_preds[i][3])
end

Accuracy on Test Data: 0.9473684210526315

Wrong Predictions :
Actual : 1  Prediction : 2  Probability Scores :[0.0659060058795523, 0.3852927659165831, 0.5488012282038646]
Actual : 1  Prediction : 2  Probability Scores :[0.0752304762891294, 0.35593520364135167, 0.5688343200695188]


### Train Accuracy & Wrong Predictions

In [28]:
train_labels = df.label[1:index];
train_preds = likelihood_reconstructor(vec(final_weights))(X_train)
train_acc, wrong_preds = accuracy(train_labels, train_preds)
println("Accuracy on Train Data: ", train_acc)

println("\nWrong Predictions :")
for i in 1:length(wrong_preds)
    
    println("Actual : ", wrong_preds[i][1], "  Prediction : ", wrong_preds[i][2], "  Probability Scores :", wrong_preds[i][3])
end

Accuracy on Train Data: 0.9553571428571429

Wrong Predictions :
Actual : 1  Prediction : 2  Probability Scores :[0.07714159801628445, 0.45340948997830377, 0.4694489120054119]
Actual : 1  Prediction : 2  Probability Scores :[0.06994160482827576, 0.41298632346593583, 0.5170720717057884]
Actual : 1  Prediction : 2  Probability Scores :[0.09609665769109921, 0.3854050138717216, 0.5184983284371791]
Actual : 1  Prediction : 2  Probability Scores :[0.10127165739935705, 0.4213696189050848, 0.47735872369555826]
Actual : 1  Prediction : 2  Probability Scores :[0.05374487672954314, 0.2870485390604867, 0.6592065842099702]


### Variational Inference

In [19]:
start_t = now()

advi = ADVI(20, 100)
q = vi(TuringModel(likelihood_reconstructor, weight_prior, X_train , y_train), advi)

end_t = now()
time_taken = (end_t - start_t) 
println("Total  Time taken: ", Dates.value(time_taken)/1000, " seconds")

┌ Info: [ADVI] Should only be seen once: optimizer created for θ
│   objectid(θ) = 13222727648826396137
└ @ AdvancedVI C:\Users\khaitan\.julia\packages\AdvancedVI\W2zsz\src\AdvancedVI.jl:202
[ADVI] Optimizing... 100% Time: 0:16:09


Total  Time taken: 984.582 seconds


### Predictions

In [20]:
### Pick "n" samples od weights and get the predictions

function prediction_vi(q, X, n_samples = 100)

    preds = []

    for i in 1:n_samples

        sampled_weight = rand(q.dist)
        test_preds = likelihood_reconstructor(vec(sampled_weight))(X)
        append!(preds, [[argmax(test_preds[i].p) - 1 for i in 1:length(test_preds)]])
    
    end
    
    pred_matrix = reduce(vcat,transpose.(preds));
    #println("Size of the prediction matrix:", size(pred_matrix))
    pred_out = [mode(pred_matrix[:, i]) for i in 1:size(pred_matrix, 2)];
    
    return pred_out
    
end

prediction_vi (generic function with 2 methods)

### Accuracy

In [21]:
### Function to compute accuracy and get the wrong predictions

function accuracy_vi(labels, predictions)
    
    acc = 0
    wrong_predictions = []

    for i in 1:length(labels)

        if labels[i] == predictions[i]
            acc += 1
        else
            #println("Wong Prediction - Actual: ", int(labels[i]), "  ---  ", "Prediction: ", predictions[i])
            append!(wrong_predictions, [[int(labels[i]), argmax(predictions[i]) - 1]])
        end
    end

    acc = acc / length(labels)
    
    return acc, wrong_predictions
end

accuracy_vi (generic function with 1 method)

### Test Accuracy & Wrong Predictions

In [23]:
test_labels = df.label[index + 1:size(df, 1)];
test_preds = prediction_vi(q, X_test)
test_acc, wrong_preds = accuracy_vi(test_labels, test_preds)
println("Accuracy on Test Data: ", test_acc)

println("\nWrong Predictions :")
for i in 1:length(wrong_preds)
    
    println("Actual : ", wrong_preds[i][1], "  Prediction : ", wrong_preds[i][2])
end

Accuracy on Test Data: 0.5

Wrong Predictions :
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0


### Train Accuracy & Wrong Predictions

In [24]:
train_labels = df.label[1:index];
train_preds = prediction_vi(q, X_train);
train_acc, wrong_preds = accuracy_vi(train_labels, train_preds)
println("Accuracy on Test Data: ", train_acc)

println("\nWrong Predictions :")
for i in 1:length(wrong_preds)
    
    println("Actual : ", wrong_preds[i][1], "  Prediction : ", wrong_preds[i][2])
end

Accuracy on Test Data: 0.7410714285714286

Wrong Predictions :
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
Actual : 1  Prediction : 0
